In [15]:
%load_ext autoreload
%autoreload 2

from config import SniffBat2Dataset
import os
from ultralytics import YOLO

import torch
import decord
from decord import VideoReader
import numpy as np
import cv2
import os
from tqdm import tqdm
import pandas as pd
import pathlib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
def ir_marker_inference(model, bat_id, date, video_paths, out_dir = None):
    """
    Run inference on set of videos

    Parameters
    ----------
    bat_id : string
        bat ID

    date : string
        YYMMDD

    video_paths : dict , key = <view_name> | value = path_to_video
        Dictionary containing name of each camera view as keys and paths to video file of said camera view as values
        E.g.) {'bottom': "path/to/bottom.mp4", 'top': "path/to/top.mp4", 'side': "path/to/side.mp4"}

    out_dir : [Optional] string
        Output directory to save inference results. If out_dir == None, results are returned by the function but not saved to disk.

    Returns
    -------
    pandas DataFrame containing detections in all frames of all videos

    """

    detection_list = []
    for view in video_paths:
        fpath = video_paths[view]
        print(fpath)
        vr = VideoReader(fpath)
        video_length = len(vr)
        print(video_length)
        decord.bridge.set_bridge('torch')


        for fnum in tqdm(range(video_length)):
            frame = np.array(vr[fnum])

            res = model.predict(source=frame, rect=False, imgsz=800, verbose=False, augment=False)

            boxes = res[0].cpu().boxes.data.numpy()
            xyxy = boxes[:,:4]
            conf = boxes[:,4]
            label = boxes[:,5]

            res_df = pd.DataFrame(boxes, columns=['xmin','ymin','xmax','ymax','confidence','class'])
            num_detections = res_df.shape[0]
            res_df['frame'] = [int(fnum)]*num_detections
            res_df['path'] = [fpath]*num_detections
            res_df['view'] = [view]*num_detections
            res_df['date'] = [date]*num_detections
            res_df['bat_id'] = [bat_id]*num_detections

            dets = res_df.to_dict(orient="records")

            detection_list += dets

    df = pd.DataFrame(detection_list, columns = ['date',
                             'bat_id',
                             'view',
                             'frame',
                             'xmin',
                             'ymin',
                             'xmax',
                             'ymax',
                             'confidence',
                             'class',
                             'name',
                             'path'])

    if(out_dir):
        pathlib.Path(out_dir).mkdir(parents=True,exist_ok=True)
        df.to_csv(os.path.join(out_dir,'ir_marker_detections.csv'))

    return df


In [17]:
date = '230910'
bat_id = '32623'

In [18]:
raw_data_path = f"/home/batlab/mnt/server2/users/KevinQi/datasets/GridBat/{bat_id}/small_cage/raw/{date}"
raw_data_path = '/media/batlab/BatDrive/GridBat/data/small_cage/raw/230910'
processed_data_path = raw_data_path.replace("raw", "processed")
video_paths = SniffBat2Dataset.get_video_paths(os.path.join(raw_data_path,'cameras'))

['Basler_acA2040-55um__24260140__20230910_152706385.mp4']
['Basler_acA2040-55um__23584894__20230910_152705354.mp4']
['Basler_a2A1920-160umBAS__40243954__20230910_152702270.mp4']
['Basler_acA800-510um__23521662__20230910_152710140.mp4']
['Basler_acA800-510um__23354674__20230910_152707517.mp4']
['Basler_a2A1920-160umBAS__40243958__20230910_152703353.mp4']
['Basler_acA800-510um__23521660__20230910_152708733.mp4']
['Basler_a2A1920-160umBAS__40243959__20230910_152704276.mp4']


In [19]:
"""
Load yolov5 weights. Download pretrained weights from lab server or train your own.
"""

yolo_weights_path = '/media/batlab/BatDrive/yolov8_batlab/detection/ir_marker_detection_v3/weights'
if(os.path.exists(f'{yolo_weights_path}/best.engine')):
    model = YOLO(f'{yolo_weights_path}/best.engine', task='detect') # TODO: Wait for yolov8 fix for best.engine
else:
    model = YOLO(f'{yolo_weights_path}/best.pt') # TODO: Wait for yolov8 fix for best.engine
    success = model.export(format="engine", device=0)  # export the model to ONNX format

In [22]:
model

In [28]:
df = ir_marker_inference(model,date,bat_id,video_paths,out_dir=raw_data_path.replace('raw', 'processed'))

/media/batlab/BatDrive/GridBat/data/small_cage/raw/230910/cameras/bottom1/Basler_acA2040-55um__24260140__20230910_152706385.mp4
302100


100%|█| 302100/302100 [2:12:56<00:00, 


/media/batlab/BatDrive/GridBat/data/small_cage/raw/230910/cameras/bottom2/Basler_acA2040-55um__23584894__20230910_152705354.mp4
302100


100%|█| 302100/302100 [1:57:24<00:00, 


/media/batlab/BatDrive/GridBat/data/small_cage/raw/230910/cameras/front1/Basler_a2A1920-160umBAS__40243954__20230910_152702270.mp4
302100


...

100%|█| 302100/302100 [1:57:19<00:00, 


/media/batlab/BatDrive/GridBat/data/small_cage/raw/230910/cameras/left1/Basler_acA800-510um__23521660__20230910_152708733.mp4
302100


100%|█| 302100/302100 [2:03:22<00:00, 


/media/batlab/BatDrive/GridBat/data/small_cage/raw/230910/cameras/left2/Basler_a2A1920-160umBAS__40243959__20230910_152704276.mp4
302100


100%|█| 302100/302100 [1:57:08<00:00, 
